In [ ]:
import os
import csv
import numpy as np
import tensorflow as tf

from model.seq2seq.Seq2Seq import *
from model.seq2seq_attn.Seq2Seq_Attn import *
from utils.utils import *

In [ ]:
# Determine what kind of model type we will use
# e.g. seq2seq, seq2seq_attn
model_type = 'seq2seq_attn'
attn_type = 'luong'
method = 'dot'

# GPU:1 allocation
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
@tf.function
def loss_function(true, pred, loss_obj):
    mask = tf.math.logical_not(tf.math.equal(true, 0))

    loss = loss_obj(true, pred)

    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask
    
    return tf.reduce_mean(loss)

In [ ]:
def train():
    # Load data
    dataset = load_dataset(data_dir)
    
    num_batches_per_epoch = len(dataset) // batch_size
    
    # Load tokenizer
    enc_tokenizer = load_tokenizer('enc-tokenizer', (x for x, y in dataset), target_vocab_size=2**13)
    dec_tokenizer = load_tokenizer('dec-tokenizer', (y for x, y in dataset), target_vocab_size=2**13)
    enc_vocab_size = enc_tokenizer.vocab_size + 1
    dec_vocab_size = dec_tokenizer.vocab_size + 2
    print(f'enc_vocab_size: {enc_vocab_size}\tdec_vocab_size: {dec_vocab_size}')
    
    # Define the optimizer and the loss function
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    
    if model_type == 'seq2seq':
        # Define the seq2seq model
        model = Seq2Seq(batch_size=batch_size, 
                        enc_max_len=enc_max_len+1, 
                        dec_max_len=dec_max_len+2, 
                        enc_unit=enc_unit, 
                        dec_unit=dec_unit, 
                        embed_dim=embed_dim, 
                        dropout_rate=dropout_rate,
                        enc_vocab_size=enc_vocab_size, 
                        dec_vocab_size=dec_vocab_size, 
                        dec_sos_token=dec_tokenizer.vocab_size)

        # Set a checkpoint directory
        checkpoint_dir = 'checkpoint/daily-korean/seq2seq'
        
    elif model_type == 'seq2seq_attn':
        # Define the seq2seq model
        model = AttnSeq2Seq(batch_size=batch_size, 
                            enc_max_len=enc_max_len+1, 
                            dec_max_len=dec_max_len+2, 
                            enc_unit=enc_unit, 
                            dec_unit=dec_unit, 
                            embed_dim=embed_dim, 
                            dropout_rate=dropout_rate,
                            enc_vocab_size=enc_vocab_size, 
                            dec_vocab_size=dec_vocab_size, 
                            dec_sos_token=dec_tokenizer.vocab_size,
                            attn_type=attn_type,
                            method=method)
        
        # Set a checkpoint directory
        checkpoint_dir = 'checkpoint/daily-korean/seq2seq_{}_attn'.format(attn_type)
        
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
        
    checkpoint_prefix = os.path.join(checkpoint_dir, 'checkpoint')
    checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)
    
    epoch_loss = tf.keras.metrics.Mean()
    
    for epoch in range(epochs):
        epoch_loss.reset_states()
        
        train_batches = batch_dataset(dataset, batch_size, enc_tokenizer, dec_tokenizer, enc_max_len, dec_max_len)
        
        for batch_idx, (batch_x, batch_y) in enumerate(train_batches):
            loss = 0.
            with tf.GradientTape() as tape:
                outputs = model(batch_x, batch_y, True)
                
                if model_type == 'seq2seq':
                    preds = outputs
                elif model_type == 'seq2seq_attn':
                    preds = outputs[0]
                    attn_weights = outputs[1]
                    
                loss = loss_function(batch_y[:, 1:], preds, loss_obj)
            
            variables = model.trainable_variables
            gradients = tape.gradient(loss, variables)
            optimizer.apply_gradients(zip(gradients, variables))
            
            epoch_loss(loss)
            
            if (batch_idx + 1) % log_interval == 0:
                print(f'[Epoch {epoch + 1}|Step {batch_idx + 1}/{num_batches_per_epoch}] loss: {loss.numpy()} (Avg. {epoch_loss.result()})')
        
        model.save_weights(filepath=checkpoint_prefix)
    
    print("Training is Done.")

In [ ]:
if __name__ == '__main__':
    np.random.seed(1234)
    tf.random.set_seed(1234)
    
    train()